<a href="https://colab.research.google.com/github/gulabpatel/LLMs/blob/main/Mistral/Finetuning_Mistral_7b_Using_AutoTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning Mistral 7b with AutoTrain

Setup Runtime
For fine-tuning Llama, a GPU instance is essential. Follow the directions below:

- Go to `Runtime` (located in the top menu bar).
- Select `Change Runtime Type`.
- Choose `T4 GPU` (or a comparable option).

### Step 1: Setup Environment

In [1]:
!pip install pandas autotrain-advanced -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 83.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 30.4 MB/s eta 0

In [2]:
!autotrain setup --update-torch

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


## Step 2: Connect to HuggingFace for Model Upload

### Logging to Hugging Face
To make sure the model can be uploaded to be used for Inference, it's necessary to log in to the Hugging Face hub.

### Getting a Hugging Face token
Steps:

1. Navigate to this URL: https://huggingface.co/settings/tokens
2. Create a write `token` and copy it to your clipboard
3. Run the code below and enter your `token`

In [3]:
from huggingface_hub import notebook_login
notebook_login()

## Step 3: Upload your dataset

Add your data set to the root directory in the Colab under the name train.csv. The AutoTrain command will look for your data there under that name.

#### Don't have a data set and want to try finetuning on an example data set?
If you don't have a dataset you can run these commands below to get an example data set and save it to train.csv

In [4]:
!git clone https://github.com/joshbickett/finetune-llama-2.git
%cd finetune-llama-2
%mv train.csv ../train.csv
%cd ..

Cloning into 'finetune-llama-2'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 70 (delta 38), reused 48 (delta 19), pack-reused 0
Receiving objects: 100% (70/70), 25.13 KiB | 2.28 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/finetune-llama-2
/content


In [5]:
import pandas as pd
df = pd.read_csv("train.csv")
df

,Concept,Funny Description Prompt,text
0,A cactus at a dance party,"A cactus, wearing disco lights and surrounded ...",###Human:\nGenerate a midjourney prompt for A ...
1,A robot on a first date,"A robot, with a bouquet of USB cables, nervous...",###Human:\nGenerate a midjourney prompt for A ...
2,A snail at a speed contest,"A snail, with a mini rocket booster, confident...",###Human:\nGenerate a midjourney prompt for A ...
3,A penguin at a beach party,"A penguin, with sunscreen and a surfboard, try...",###Human:\nGenerate a midjourney prompt for A ...
4,A cloud in a bad mood,"A cloud, grumbling and dropping mini lightning...",###Human:\nGenerate a midjourney prompt for A ...
...,...,...,...
112,A donut feeling the hole emptiness,"A donut, in existential bakery therapy, ponder...",###Human:\nGenerate a midjourney prompt for A ...
113,A pineapple with a prickly attitude,"A pineapple, in a prickly personality class, s...",###Human:\nGenerate a midjourney prompt for A ...
114,A calculator crunching life's problems,"A calculator, at a problem-solving workshop, c...",###Human:\nGenerate a midjourney prompt for A ...
115,A kite reaching new heights,"A kite, in an altitude adjustment session, unt...",###Human:\nGenerate a midjourney prompt for A ...


In [6]:
df['text'][15]

'###Human:\nGenerate a midjourney prompt for A book on a mystery adventure\n\n###Assistant:\nA book, wearing detective glasses, flipping through its own pages, trying to solve the cliffhanger it was left on.'

## Step 4: Overview of AutoTrain command

#### Short overview of what the command flags do.

- `!autotrain`: Command executed in environments like a Jupyter notebook to run shell commands directly. `autotrain` is an automatic training utility.

- `llm`: A sub-command or argument specifying the type of task

- `--train`: Initiates the training process.

- `--project_name`: Sets the name of the project

- `--model abhishek/llama-2-7b-hf-small-shards`: Specifies original model that is hosted on Hugging Face named "llama-2-7b-hf-small-shards" under the "abhishek".

- `--data_path .`: The path to the dataset for training. The "." refers to the current directory. The `train.csv` file needs to be located in this directory.

- `--use_int4`: Use of INT4 quantization to reduce model size and speed up inference times at the cost of some precision.

- `--learning_rate 2e-4`: Sets the learning rate for training to 0.0002.

- `--train_batch_size 12`: Sets the batch size for training to 12.

- `--num_train_epochs 3`: The training process will iterate over the dataset 3 times.

### Steps needed before running
Go to the `!autotrain` code cell below and update it by following the steps below:

1. After `--project_name` replace `*enter-a-project-name*` with the name that you'd like to call the project
2. After `--repo_id` replace `*username*/*repository*`. Replace `*username*` with your Hugging Face username and `*repository*` with the repository name you'd like it to be created under. You don't need to create this repository before hand, it will automatically be created and uploaded once the training is completed.
3. Confirm that `train.csv` is in the root directory in the Colab. The `--data_path .` flag will make it so that AutoTrain looks for your data there.
4. Make sure to add the LoRA Target Modules to be trained `--target-modules q_proj, v_proj`
5. Once you've made these changes you're all set, run the command below!

In [7]:
!autotrain llm --train --project_name mistral-7b-mj-finetuned --model bn22/Mistral-7B-Instruct-v0.1-sharded --data_path . --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 12 --num_train_epochs 3 --trainer sft --target_modules q_proj,v_proj --push_to_hub --repo_id ashishpatel26/mistral-7b-mj-finetuned

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='.', train_split='train', valid_split=None, text_column='text', model='bn22/Mistral-7B-Instruct-v0.1-sharded', learning_rate=0.0002, num_train_epochs=3, train_batch_size=12, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='mistral-7b-mj-finetuned', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=True, use_int8=False, model_max_length=1024, repo_id='ashishpatel26/mistral-7b-mj-finetuned', use_int4=True, trainer='sft', target_modules='q_proj,v_proj', merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=False, func=<function run_llm_command_factory

## Step 5: Completed 🎉
After the command above is completed your Model will be uploaded to Hugging Face.

#### Learn more about AutoTrain (optional)
If you want to learn more about what command-line flags are available

## Step 6: Inference Engine

In [8]:
!autotrain llm -h

usage: autotrain <command> [<args>] llm
       [-h]
       [--train]
       [--deploy]
       [--inference]
       [--data_path DATA_PATH]
       [--train_split TRAIN_SPLIT]
       [--valid_split VALID_SPLIT]
       [--text_column TEXT_COLUMN]
       [--model MODEL]
       [--learning_rate LEARNING_RATE]
       [--num_train_epochs NUM_TRAIN_EPOCHS]
       [--train_batch_size TRAIN_BATCH_SIZE]
       [--warmup_ratio WARMUP_RATIO]
       [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
       [--optimizer OPTIMIZER]
       [--scheduler SCHEDULER]
       [--weight_decay WEIGHT_DECAY]
       [--max_grad_norm MAX_GRAD_NORM]
       [--seed SEED]
       [--add_eos_token]
       [--block_size BLOCK_SIZE]
       [--use_peft]
       [--lora_r LORA_R]
       [--lora_alpha LORA_ALPHA]
       [--lora_dropout LORA_DROPOUT]
       [--logging_steps LOGGING_STEPS]
       [--project_name PROJECT_NAME]
       [--evaluation_strategy EVALUATION_STRATEGY]
       [--save_total_limit SAVE_TOTAL_LIM

In [9]:
!pip install -q peft  accelerate bitsandbytes safetensors

In [10]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
adapters_name = "ashishpatel26/mistral-7b-mj-finetuned"
model_name = "bn22/Mistral-7B-Instruct-v0.1-sharded" #"mistralai/Mistral-7B-Instruct-v0.1"


device = "cuda" # the device to load the model onto

In [11]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

## Step 7: Peft Model Loading with upload model

In [13]:
model = PeftModel.from_pretrained(model, adapters_name)

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Successfully loaded the model bn22/Mistral-7B-Instruct-v0.1-sharded into memory


In [15]:
text = "[INST] generate a midjourney prompt for A person walks in the rain [/INST]"

encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded
model.to(device)
generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST] generate a midjourney prompt for A person walks in the rain [/INST] A person walks in the rain, but as they approach the center of the city, they notice that the rain has stopped and the sun is shining brightly. Feeling torn between their duty and the safety of their family, they must decide what to do next. Will they continue on their mission, risking their own safety and that of their loved ones? Or will they turn back, leaving their mission unfinished but ensuring their own safety?</s>
